In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use("dark_background")

import datetime
import os
import shutil
from collections import Counter
from pprint import pprint

import seaborn as sn
from tqdm import tqdm

rs = 42

In [31]:
# calls_path="/kaggle/input/dataset/calls.csv"
# customer_path="/kaggle/input/dataset/customers.csv"
# reason_path="/kaggle/input/dataset/reason.csv"
# sentiment_stat="/kaggle/input/dataset/sentiment_statistic.csv"
# test_path="/kaggle/input/dataset/test.csv"

calls_path = "DataSet/callsf0d4f5a.csv"
customers_path = "DataSet/customers2afd6ea.csv"
reasons_path = "DataSet/reason18315ff.csv"
sentiments_path = "DataSet/sentiment_statisticscc1e57a.csv"
test_path = "DataSet/testbc7185d.csv"

## Merging DataFrames

In [32]:
Data_calls = pd.read_csv(calls_path)
Data_reasons = pd.read_csv(reasons_path)
Data_customers = pd.read_csv(customers_path)
Data_sentiments = pd.read_csv(sentiments_path)

In [33]:
Data_customers.dropna(axis = 0,inplace=True,ignore_index=True)
Data_calls.drop('call_transcript',axis = 1,inplace=True)
Data_customers.drop('customer_name',axis = 1,inplace=True)

In [34]:
Data_merged = pd.merge(Data_calls,Data_customers,how='inner',on = 'customer_id')
Data_merged.sample(5)

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,elite_level_code
1623,3246257574,6514452723,860774,8/2/2024 18:40,8/2/2024 18:43,8/2/2024 18:47,1.0
12063,7566212527,8726104228,878048,8/10/2024 0:31,8/10/2024 0:40,8/10/2024 0:42,0.0
487,1483057112,2544367371,391212,8/1/2024 14:19,8/1/2024 14:34,8/1/2024 14:43,4.0
10052,9429640941,2765468434,171021,8/7/2024 15:44,8/7/2024 15:49,8/7/2024 15:51,0.0
838,8769406875,607691023,158319,8/2/2024 0:33,8/2/2024 0:41,8/2/2024 0:46,2.0


In [35]:
Data_merged = pd.merge(Data_merged,Data_reasons,how='inner',on = 'call_id')
Data_merged.sample(5)

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,elite_level_code,primary_call_reason
6672,1923241284,5271315641,731015,8/4/2024 16:46,8/4/2024 16:52,8/4/2024 16:57,1.0,Products and Services
40254,7969700571,6525826654,459044,8/31/2024 7:57,8/31/2024 8:06,8/31/2024 8:11,3.0,Post-Flight
8124,1208697190,2869867567,131036,8/6/2024 5:53,8/6/2024 6:00,8/6/2024 6:07,3.0,Mileage Plus
10123,4628575395,4927473843,126555,8/8/2024 15:38,8/8/2024 15:47,8/8/2024 15:50,0.0,Seating
11029,7195275141,3578227379,126555,8/9/2024 17:36,8/9/2024 17:44,8/9/2024 18:36,0.0,ETC


In [36]:
Data_merged = pd.merge(Data_merged,Data_sentiments,how='inner',on = ['call_id','agent_id'])
Data_merged.sample(5)

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,elite_level_code,primary_call_reason,agent_tone,customer_tone,average_sentiment,silence_percent_average
33817,8039662420,6401861670,698596,8/25/2024 9:15,8/25/2024 9:20,8/25/2024 9:25,1.0,IRROPS,neutral,polite,-0.05,0.27
25634,43212344,7090044551,668841,8/18/2024 16:05,8/18/2024 16:10,8/18/2024 16:32,0.0,Check-In,frustrated,polite,-0.29,0.39
28805,6359436348,1583658943,213093,8/21/2024 22:14,8/21/2024 22:21,8/21/2024 22:30,0.0,Mileage Plus,calm,polite,0.20,0.13
28539,3741962655,7094042559,251287,8/21/2024 14:47,8/21/2024 14:54,8/21/2024 14:59,3.0,Baggage,calm,neutral,0.08,0.41
19751,2048687161,5328155311,542034,8/15/2024 17:09,8/15/2024 17:13,8/15/2024 17:58,0.0,Communications,calm,calm,0.04,0.62


In [37]:
# Assuming 'category_column' is the name of the column with the categorical variables
Data_merged["primary_call_reason"] = (
    Data_merged["primary_call_reason"]
    .str.strip()
    .str.replace(r"\s+|-|&|and", " ", regex=True)
)
Data_merged.sample(5)

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,elite_level_code,primary_call_reason,agent_tone,customer_tone,average_sentiment,silence_percent_average
7221,4609621083,1057718379,683244,8/4/2024 21:50,8/4/2024 22:00,8/4/2024 22:27,0.0,Voluntary Change,calm,frustrated,0.06,0.77
33130,4609177372,7071920725,400977,8/24/2024 22:41,8/24/2024 22:47,8/24/2024 22:51,0.0,Upgrade,calm,frustrated,0.03,0.05
39711,7949462654,514709451,981776,8/30/2024 17:09,8/30/2024 17:15,8/30/2024 18:02,2.0,Mileage Plus,neutral,frustrated,-0.01,0.40
29599,8417798480,4725597649,198686,8/22/2024 21:38,8/22/2024 21:47,8/22/2024 21:55,4.0,Post Flight,neutral,calm,-0.08,0.18
30624,2751481707,7824927699,470164,8/24/2024 4:19,8/24/2024 4:26,8/24/2024 4:28,2.0,Voluntary Change,calm,polite,0.29,0.17


In [38]:
Data_merged_2 = pd.get_dummies(
    Data_merged, columns=["primary_call_reason",'customer_tone','agent_tone','elite_level_code'], dtype="int"
)
Data_merged_2.sample(5)

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,average_sentiment,silence_percent_average,primary_call_reason_Baggage,primary_call_reason_Booking,...,agent_tone_calm,agent_tone_frustrated,agent_tone_neutral,agent_tone_polite,elite_level_code_0.0,elite_level_code_1.0,elite_level_code_2.0,elite_level_code_3.0,elite_level_code_4.0,elite_level_code_5.0
30965,5955419983,3013404028,993862,8/24/2024 8:48,8/24/2024 8:57,8/24/2024 9:17,0.09,0.37,0,0,...,1,0,0,0,0,0,1,0,0,0
17974,9313754586,7479241551,192245,8/13/2024 11:18,8/13/2024 11:23,8/13/2024 11:30,-0.04,0.06,0,0,...,0,0,1,0,0,1,0,0,0,0
24749,4201088069,4491467515,887241,8/18/2024 11:52,8/18/2024 11:57,8/18/2024 12:11,0.08,0.58,0,0,...,1,0,0,0,1,0,0,0,0,0
20787,1379099740,2127892604,519057,8/17/2024 1:48,8/17/2024 1:57,8/17/2024 2:04,0.12,0.11,0,0,...,1,0,0,0,0,1,0,0,0,0
14283,2492046748,527401081,828869,8/11/2024 4:56,8/11/2024 5:03,8/11/2024 5:10,-0.26,0.52,0,0,...,0,1,0,0,0,0,0,1,0,0


In [39]:
# Convert the columns to datetime format
Data_merged_2["call_start_datetime"] = pd.to_datetime(
    Data_merged_2["call_start_datetime"], format="%m/%d/%Y %H:%M"
)
Data_merged_2["agent_assigned_datetime"] = pd.to_datetime(
    Data_merged_2["agent_assigned_datetime"], format="%m/%d/%Y %H:%M"
)
Data_merged_2["call_end_datetime"] = pd.to_datetime(
    Data_merged_2["call_end_datetime"], format="%m/%d/%Y %H:%M"
)

# Calculate wait_time (in minutes) and call_duration (in minutes)
Data_merged_2["wait_time"] = (
    Data_merged_2["agent_assigned_datetime"] - Data_merged_2["call_start_datetime"]
).dt.total_seconds() / 60

Data_merged_2["call_duration"] = (
    Data_merged_2["call_end_datetime"] - Data_merged_2["agent_assigned_datetime"]
).dt.total_seconds() / 60

# Create a new dataframe with the original columns plus the new calculated columns
Data_merged_2 = Data_merged_2.copy()
Data_merged_2.sample(5)

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,average_sentiment,silence_percent_average,primary_call_reason_Baggage,primary_call_reason_Booking,...,agent_tone_neutral,agent_tone_polite,elite_level_code_0.0,elite_level_code_1.0,elite_level_code_2.0,elite_level_code_3.0,elite_level_code_4.0,elite_level_code_5.0,wait_time,call_duration
30950,6288020070,1482097366,575131,2024-08-24 08:45:00,2024-08-24 08:55:00,2024-08-24 08:59:00,-0.21,0.23,0,0,...,1,0,1,0,0,0,0,0,10.0,4.0
13383,1241138296,778248633,650595,2024-08-10 16:45:00,2024-08-10 16:52:00,2024-08-10 16:54:00,-0.18,0.05,0,0,...,1,0,0,1,0,0,0,0,7.0,2.0
2314,1695901249,6232990479,357288,2024-08-03 09:41:00,2024-08-03 09:46:00,2024-08-03 09:51:00,-0.09,0.16,0,0,...,1,0,1,0,0,0,0,0,5.0,5.0
36936,7918289876,4286018414,734708,2024-08-27 09:03:00,2024-08-27 09:11:00,2024-08-27 09:28:00,-0.07,0.30,0,0,...,1,0,0,0,0,0,1,0,8.0,17.0
41565,1097931976,5500228060,153182,2024-08-31 14:22:00,2024-08-31 14:30:00,2024-08-31 14:58:00,-0.16,0.53,0,0,...,1,0,0,1,0,0,0,0,8.0,28.0


In [40]:
Data_merged_2.drop(['call_id','customer_id','agent_id','call_start_datetime','agent_assigned_datetime','call_end_datetime'],axis = 1,inplace = True)
Data_merged_2.sample(5)

,average_sentiment,silence_percent_average,primary_call_reason_Baggage,primary_call_reason_Booking,primary_call_reason_Check In,primary_call_reason_Checkout,primary_call_reason_Communications,primary_call_reason_Digital Support,primary_call_reason_Disability,primary_call_reason_ETC,...,agent_tone_neutral,agent_tone_polite,elite_level_code_0.0,elite_level_code_1.0,elite_level_code_2.0,elite_level_code_3.0,elite_level_code_4.0,elite_level_code_5.0,wait_time,call_duration
5269,0.03,0.46,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,10.0,6.0
30715,0.11,0.33,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,7.0,6.0
19197,-0.20,0.26,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,4.0,6.0
1175,-0.07,0.11,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,12.0,10.0
462,0.05,0.08,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,3.0,1.0


## Train Test Split

In [41]:
from sklearn.model_selection import train_test_split

In [100]:
X = Data_merged_2.values[:,:-2]
Y = Data_merged_2.values[:,-2:]
X.shape,Y.shape

((42694, 38), (42694, 2))

In [101]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state = rs,test_size = 0.2)
X_train.shape ,X_test.shape ,Y_train.shape,Y_test.shape

((34155, 38), (8539, 38), (34155, 2), (8539, 2))

In [102]:
from sklearn.preprocessing import MinMaxScaler

In [103]:
scaler = MinMaxScaler() 
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test)

## Training Model

In [104]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score

### > Training Ensemble Random Forest Model for Predicting waiting time

In [105]:
my_model1 = RandomForestRegressor(max_leaf_nodes=300)
my_model1.fit(X_train,Y_train[:,0])

RandomForestRegressor(max_leaf_nodes=300)

In [106]:
Y_pred = my_model1.predict(X_test)
error = mean_squared_error(Y_test[:,0],Y_pred)
score = r2_score(Y_test[:,0],Y_pred)
error ,score

(1.6645519799516575, 0.7329410756597061)

#### So We are certaninly able to predict average wait time that Can help us keeping track of how much more time we need to reach to a customer . and will give him/her the predicted time for their turn to come.

### > Training Ensemble Random Forest Model for Predicting Handle time

In [107]:
my_model2 = RandomForestRegressor(max_leaf_nodes=300)
my_model2.fit(X_train,Y_train[:,1])

RandomForestRegressor(max_leaf_nodes=300)

In [108]:
Y_pred = my_model2.predict(X_test)
error = mean_squared_error(Y_test[:,1],Y_pred)
score = r2_score(Y_test[:,1],Y_pred)
error ,score

(122.88196014999346, 0.2537522316161138)

#### We are not getting that good result in predicting the handle time .